# ControlD API tool use

Claude tool use with ControlD API endpoints.

In [25]:
from anthropic import Anthropic
import requests
from typing import Dict, List, Optional
import os
from datetime import date
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Anthropic client with API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_api_key:
    raise ValueError("Please set ANTHROPIC_API_KEY environment variable")
    
client = Anthropic(api_key=anthropic_api_key)
MODEL_NAME = "claude-3-5-sonnet-20241022"

In [26]:
def redact_sensitive_info(text: str, auth_token: str) -> str:
    """Redact sensitive information from output"""
    if not isinstance(text, str):
        text = str(text)
        
    # Redact full token
    if auth_token in text:
        text = text.replace(auth_token, "[REDACTED_TOKEN]")
    
    # Redact token in Bearer format
    text = text.replace(f"Bearer {auth_token}", "Bearer [REDACTED]")
    
    # Redact token in api.* format
    if "api." in text:
        text = text.replace(text[text.find("api."):text.find("api.") + 71], "[REDACTED_API_TOKEN]")
    
    return text

In [27]:
def get_controld_profiles(auth_token: str) -> Dict:
    """List all profiles"""
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}'
    }
    
    response = requests.get(
        'https://api.controld.com/profiles',
        headers=headers
    )
    
    response.raise_for_status()
    return response.json()

def create_controld_profile(auth_token: str, name: str, clone_profile_id: Optional[str] = None) -> Dict:
    """Create a new profile or clone an existing one"""
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}',
        'content-type': 'application/x-www-form-urlencoded'
    }
    
    data = {'name': name}
    if clone_profile_id:
        data['clone_profile_id'] = clone_profile_id
    
    # Debug logging with redaction
    print("\nRequest Details:")
    print(f"URL: https://api.controld.com/profiles")
    print(f"Method: POST")
    print(f"Headers: {redact_sensitive_info(str(headers), auth_token)}")
    print(f"Data: {data}")
    
    response = requests.post(
        'https://api.controld.com/profiles',
        headers=headers,
        data=data
    )
    
    # Debug logging with redaction
    print("\nResponse Details:")
    print(f"Status Code: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    print(f"Response Body: {redact_sensitive_info(response.text, auth_token)}")
    
    try:
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"\nError: {str(e)}")
        return {"error": str(e)}

# Add this function with your other API functions
def get_controld_filters(auth_token: str, profile_id: str) -> Dict:
    """List all native filters for a specific profile
    
    Args:
        auth_token (str): ControlD API authentication token
        profile_id (str): Primary key (PK) of the profile
        
    Returns:
        Dict: JSON response containing filter information
    """
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}'
    }
    
    response = requests.get(
        f'https://api.controld.com/profiles/{profile_id}/filters',
        headers=headers
    )
    
    response.raise_for_status()
    return response.json()

In [28]:
tools = [
    {
        "name": "get_profiles",
        "description": "Retrieves all profiles associated with a ControlD account",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                }
            },
            "required": ["auth_token"]
        }
    },
    {
        "name": "create_profile",
        "description": "Creates a new ControlD profile or clones an existing one",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                },
                "name": {
                    "type": "string",
                    "description": "Name of the new profile"
                },
                "clone_profile_id": {
                    "type": "string",
                    "description": "The exact profile ID (PK) to clone from. Must be the full ID string, not just the profile name."
                }
            },
            "required": ["auth_token", "name"]
        }
    },
    {
        "name": "get_filters",
        "description": "Retrieves all native filters for a specific ControlD profile",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                },
                "profile_id": {
                    "type": "string",
                    "description": "Primary key (PK) of the profile to get filters for"
                }
            },
            "required": ["auth_token", "profile_id"]
        }
    }
]

In [29]:
def chat_with_controld(user_query: str, auth_token: str):
    messages = [{
        "role": "user", 
        "content": f"Auth Token: {auth_token}\n\nUser Query: {user_query}"
    }]
    
    system_prompt = """
    You are an assistant that helps users interact with their ControlD profiles.
    You have access to the following tools:
    1. get_profiles - Lists all existing profiles
    2. create_profile - Creates a new profile or clones an existing one
    3. get_filters - Lists all native filters for a specific profile
    
    When users want to see filters for a profile:
    1. First use get_profiles to get the profile ID if needed
    2. Then use get_filters with the profile ID to retrieve filter information
    
    When users want to create a profile and clone it from an existing one:
    1. First use get_profiles to confirm the source profile exists
    2. Then use create_profile with both the name and clone_profile_id parameters
    
    Do not just describe what you'll do - actually make the API calls using the tools.
    """
    
    # Get initial profile information
    response = client.messages.create(
        system=system_prompt,
        model=MODEL_NAME,
        max_tokens=1000,
        messages=messages,
        tools=tools,
    )
    
    last_content_block = response.content[-1]
    if last_content_block.type == "tool_use":
        try:
            if last_content_block.name == "get_profiles":
                profiles = get_controld_profiles(auth_token)
                print("\nVerifying profiles:", redact_sensitive_info(str(profiles), auth_token))
                
                # Add profile data to conversation
                create_messages = [
                    *messages,
                    {"role": "assistant", "content": "Retrieved profile information."},
                    {"role": "user", "content": f"Profiles data: {profiles}"}
                ]
                
                # Get next action from Claude
                create_response = client.messages.create(
                    system=system_prompt,
                    model=MODEL_NAME,
                    max_tokens=1000,
                    messages=create_messages,
                    tools=tools,
                )
                
                next_block = create_response.content[-1]
                if next_block.type == "tool_use":
                    if next_block.name == "create_profile":
                        result = create_controld_profile(
                            auth_token=auth_token,
                            name=next_block.input.get("name"),
                            clone_profile_id=next_block.input.get("clone_profile_id")
                        )
                        print("\nCreate Profile Result:", redact_sensitive_info(str(result), auth_token))
                    elif next_block.name == "get_filters":
                        result = get_controld_filters(
                            auth_token=auth_token,
                            profile_id=next_block.input.get("profile_id")
                        )
                        print("\nProfile Filters:", redact_sensitive_info(str(result), auth_token))
            
            elif last_content_block.name == "get_filters":
                result = get_controld_filters(
                    auth_token=auth_token,
                    profile_id=last_content_block.input.get("profile_id")
                )
                print("\nProfile Filters:", redact_sensitive_info(str(result), auth_token))
            
            elif last_content_block.name == "create_profile":
                result = create_controld_profile(
                    auth_token=auth_token,
                    name=last_content_block.input.get("name"),
                    clone_profile_id=last_content_block.input.get("clone_profile_id")
                )
                print("\nCreate Profile Result:", redact_sensitive_info(str(result), auth_token))
                
        except Exception as e:
            print(f"\nError executing tool: {type(e).__name__}: {str(e)}")
            import traceback
            print(traceback.format_exc())

In [30]:
# Get auth token
controld_token = os.getenv("CONTROLD_API_TOKEN")
if not controld_token:
    raise ValueError("Please set CONTROLD_API_TOKEN environment variable")

# Test queries
queries = [
    "Show me the filters for the 'arun' profile",
]

for query in queries:
    chat_with_controld(query, controld_token)


Verifying profiles: {'body': {'profiles': [{'PK': '626071rduv3k', 'updated': 1730575397, 'name': 'arun', 'profile': {'flt': {'count': 20}, 'cflt': {'count': 0}, 'ipflt': {'count': 1}, 'rule': {'count': 15}, 'svc': {'count': 65}, 'grp': {'count': 0}, 'opt': {'count': 2, 'data': [{'PK': 'ai_malware', 'value': 0.5}, {'PK': 'block_rfc1918', 'value': 1}]}, 'da': {'do': 1, 'status': 1}}}, {'PK': '626842sjcqro', 'updated': 1729986987, 'name': 'family', 'profile': {'flt': {'count': 21}, 'cflt': {'count': 0}, 'ipflt': {'count': 1}, 'rule': {'count': 2}, 'svc': {'count': 65}, 'grp': {'count': 0}, 'opt': {'count': 2, 'data': [{'PK': 'ai_malware', 'value': 0.5}, {'PK': 'block_rfc1918', 'value': 1}]}, 'da': {'do': 1, 'status': 1}}}, {'PK': '628525otprmu', 'updated': 1729134044, 'name': 'relatives', 'profile': {'flt': {'count': 21}, 'cflt': {'count': 0}, 'ipflt': {'count': 1}, 'rule': {'count': 2}, 'svc': {'count': 65}, 'grp': {'count': 0}, 'opt': {'count': 2, 'data': [{'PK': 'ai_malware', 'value':